In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Image
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop

# import tensorflow as tf
import tensorflow.compat.v1 as tf #ca sa-mi mearga read file din functie
tf.disable_v2_behavior()

from tensorflow import keras
from tensorflow.keras import layers

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# citire imagini de antrenare plus etichete de antrenare
f = open ("train.txt")
train_images = []
train_labels = []
for line in enumerate(open("train.txt", "r")):
    v = line[1].split(',')
    image_name = 'train/'+v[0]
    label = int(v[1])
    train_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    train_images.append(image)
# transform într-un array
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [5]:
# citire imagini de testare 
f = open ("test.txt")
test_images = []
for line in enumerate(open("test.txt", "r")):
    image_name = 'test/'+ line[1][:len(line[1])-1]
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    test_images.append(image)
# transform într-un array
test_images = np.array(test_images)

In [6]:
# citire imagini de validare plus etichete de validare
f = open ("validation.txt")
validation_images = []
validation_labels = []
for line in enumerate(open("validation.txt", "r")):
    v = line[1].split(',')
    image_name = 'validation/'+v[0]
    label = int(v[1])
    validation_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    validation_images.append(image)
# transform într-un array
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

In [7]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      multiple                  0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049  

In [8]:
# train the top layer
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

In [9]:
#fine tunning
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      multiple                  0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049  

In [10]:
#Neural Network Classifier 

nb_classes = 10 #10 categorii notate de la 0 la 9

x_train=train_images.astype('float32')
x_val=validation_images.astype('float32')

y_val=np_utils.to_categorical(validation_labels, nb_classes)
y_train=np_utils.to_categorical(train_labels, nb_classes)

model = Sequential()
model.add(Dense(100, input_shape =(x_train.shape[1],)))
model.add(Activation ('relu'))
model.add(Dense(200))
model.add(Activation ('sigmoid'))
model.add(Dense(nb_classes))
model.add(Activation ('softmax'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-6, decay=1e-6), metrics=['accuracy'])
history= model.fit (x_train, y_train, batch_size=200, epochs=1000, verbose=2,  validation_split=0.2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)


Train on 24000 samples, validate on 6001 samples
Epoch 1/1000


D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


24000/24000 - 1s - loss: 0.2836 - acc: 0.9026 - val_loss: 0.2440 - val_acc: 0.9098
Epoch 2/1000
24000/24000 - 1s - loss: 0.2325 - acc: 0.9124 - val_loss: 0.2193 - val_acc: 0.9164
Epoch 3/1000
24000/24000 - 1s - loss: 0.2120 - acc: 0.9188 - val_loss: 0.2051 - val_acc: 0.9218
Epoch 4/1000
24000/24000 - 1s - loss: 0.1996 - acc: 0.9230 - val_loss: 0.1944 - val_acc: 0.9243
Epoch 5/1000
24000/24000 - 1s - loss: 0.1901 - acc: 0.9258 - val_loss: 0.1903 - val_acc: 0.9256
Epoch 6/1000
24000/24000 - 1s - loss: 0.1828 - acc: 0.9287 - val_loss: 0.1789 - val_acc: 0.9314
Epoch 7/1000
24000/24000 - 1s - loss: 0.1770 - acc: 0.9315 - val_loss: 0.1774 - val_acc: 0.9309
Epoch 8/1000
24000/24000 - 1s - loss: 0.1722 - acc: 0.9329 - val_loss: 0.1810 - val_acc: 0.9307
Epoch 9/1000
24000/24000 - 1s - loss: 0.1686 - acc: 0.9344 - val_loss: 0.1710 - val_acc: 0.9326
Epoch 10/1000
24000/24000 - 1s - loss: 0.1640 - acc: 0.9362 - val_loss: 0.1742 - val_acc: 0.9307
Epoch 11/1000
24000/24000 - 1s - loss: 0.1619 - acc:

Epoch 86/1000
24000/24000 - 1s - loss: 0.0662 - acc: 0.9778 - val_loss: 0.1613 - val_acc: 0.9428
Epoch 87/1000
24000/24000 - 1s - loss: 0.0653 - acc: 0.9783 - val_loss: 0.1509 - val_acc: 0.9458
Epoch 88/1000
24000/24000 - 1s - loss: 0.0647 - acc: 0.9786 - val_loss: 0.1530 - val_acc: 0.9452
Epoch 89/1000
24000/24000 - 1s - loss: 0.0639 - acc: 0.9783 - val_loss: 0.1615 - val_acc: 0.9439
Epoch 90/1000
24000/24000 - 1s - loss: 0.0635 - acc: 0.9790 - val_loss: 0.1536 - val_acc: 0.9461
Epoch 91/1000
24000/24000 - 1s - loss: 0.0627 - acc: 0.9797 - val_loss: 0.1520 - val_acc: 0.9462
Epoch 92/1000
24000/24000 - 1s - loss: 0.0625 - acc: 0.9793 - val_loss: 0.1613 - val_acc: 0.9444
Epoch 93/1000
24000/24000 - 1s - loss: 0.0610 - acc: 0.9800 - val_loss: 0.1613 - val_acc: 0.9428
Epoch 94/1000
24000/24000 - 1s - loss: 0.0610 - acc: 0.9800 - val_loss: 0.1565 - val_acc: 0.9458
Epoch 95/1000
24000/24000 - 1s - loss: 0.0601 - acc: 0.9802 - val_loss: 0.1596 - val_acc: 0.9437
Epoch 96/1000
24000/24000 - 1s

Epoch 170/1000
24000/24000 - 1s - loss: 0.0258 - acc: 0.9940 - val_loss: 0.2293 - val_acc: 0.9405
Epoch 171/1000
24000/24000 - 1s - loss: 0.0261 - acc: 0.9938 - val_loss: 0.2203 - val_acc: 0.9420
Epoch 172/1000
24000/24000 - 1s - loss: 0.0253 - acc: 0.9941 - val_loss: 0.2218 - val_acc: 0.9420
Epoch 173/1000
24000/24000 - 1s - loss: 0.0254 - acc: 0.9941 - val_loss: 0.2054 - val_acc: 0.9452
Epoch 174/1000
24000/24000 - 1s - loss: 0.0256 - acc: 0.9936 - val_loss: 0.1997 - val_acc: 0.9467
Epoch 175/1000
24000/24000 - 1s - loss: 0.0242 - acc: 0.9944 - val_loss: 0.2163 - val_acc: 0.9419
Epoch 176/1000
24000/24000 - 1s - loss: 0.0248 - acc: 0.9943 - val_loss: 0.2012 - val_acc: 0.9470
Epoch 177/1000
24000/24000 - 1s - loss: 0.0239 - acc: 0.9946 - val_loss: 0.2017 - val_acc: 0.9465
Epoch 178/1000
24000/24000 - 1s - loss: 0.0235 - acc: 0.9945 - val_loss: 0.2088 - val_acc: 0.9446
Epoch 179/1000
24000/24000 - 1s - loss: 0.0235 - acc: 0.9946 - val_loss: 0.2352 - val_acc: 0.9390
Epoch 180/1000
24000

...

Epoch 590/1000
24000/24000 - 1s - loss: 0.0029 - acc: 0.9995 - val_loss: 0.4198 - val_acc: 0.9451
Epoch 591/1000
24000/24000 - 1s - loss: 0.0049 - acc: 0.9991 - val_loss: 0.4165 - val_acc: 0.9453
Epoch 592/1000
24000/24000 - 1s - loss: 0.0046 - acc: 0.9991 - val_loss: 0.4232 - val_acc: 0.9447
Epoch 593/1000
24000/24000 - 1s - loss: 0.0045 - acc: 0.9992 - val_loss: 0.4465 - val_acc: 0.9411
Epoch 594/1000
24000/24000 - 1s - loss: 0.0056 - acc: 0.9990 - val_loss: 0.4213 - val_acc: 0.9449
Epoch 595/1000
24000/24000 - 1s - loss: 0.0041 - acc: 0.9992 - val_loss: 0.4172 - val_acc: 0.9451
Epoch 596/1000
24000/24000 - 1s - loss: 0.0052 - acc: 0.9991 - val_loss: 0.4188 - val_acc: 0.9446
Epoch 597/1000
24000/24000 - 1s - loss: 0.0053 - acc: 0.9991 - val_loss: 0.4227 - val_acc: 0.9441
Epoch 598/1000
24000/24000 - 1s - loss: 0.0056 - acc: 0.9990 - val_loss: 0.4179 - val_acc: 0.9452
Epoch 599/1000
24000/24000 - 1s - loss: 0.0051 - acc: 0.9990 - val_loss: 0.4165 - val_acc: 0.9445
Epoch 600/1000
24000

Epoch 674/1000
24000/24000 - 1s - loss: 0.0027 - acc: 0.9995 - val_loss: 0.4427 - val_acc: 0.9451
Epoch 675/1000
24000/24000 - 1s - loss: 0.0055 - acc: 0.9992 - val_loss: 0.4411 - val_acc: 0.9448
Epoch 676/1000
24000/24000 - 1s - loss: 0.0042 - acc: 0.9992 - val_loss: 0.4473 - val_acc: 0.9435
Epoch 677/1000
24000/24000 - 1s - loss: 0.0024 - acc: 0.9996 - val_loss: 0.4408 - val_acc: 0.9449
Epoch 678/1000
24000/24000 - 1s - loss: 0.0047 - acc: 0.9992 - val_loss: 0.4438 - val_acc: 0.9445
Epoch 679/1000
24000/24000 - 1s - loss: 0.0027 - acc: 0.9996 - val_loss: 0.4452 - val_acc: 0.9450
Epoch 680/1000
24000/24000 - 1s - loss: 0.0046 - acc: 0.9991 - val_loss: 0.4410 - val_acc: 0.9447
Epoch 681/1000
24000/24000 - 1s - loss: 0.0023 - acc: 0.9996 - val_loss: 0.4484 - val_acc: 0.9445
Epoch 682/1000
24000/24000 - 1s - loss: 0.0059 - acc: 0.9990 - val_loss: 0.4450 - val_acc: 0.9446
Epoch 683/1000
24000/24000 - 1s - loss: 0.0043 - acc: 0.9992 - val_loss: 0.4435 - val_acc: 0.9438
Epoch 684/1000
24000

Epoch 758/1000
24000/24000 - 1s - loss: 0.0030 - acc: 0.9995 - val_loss: 0.4618 - val_acc: 0.9447
Epoch 759/1000
24000/24000 - 1s - loss: 0.0052 - acc: 0.9991 - val_loss: 0.4638 - val_acc: 0.9443
Epoch 760/1000
24000/24000 - 1s - loss: 0.0039 - acc: 0.9993 - val_loss: 0.4646 - val_acc: 0.9446
Epoch 761/1000
24000/24000 - 1s - loss: 0.0035 - acc: 0.9995 - val_loss: 0.4656 - val_acc: 0.9447
Epoch 762/1000
24000/24000 - 1s - loss: 0.0050 - acc: 0.9992 - val_loss: 0.4565 - val_acc: 0.9450
Epoch 763/1000
24000/24000 - 1s - loss: 0.0028 - acc: 0.9995 - val_loss: 0.4630 - val_acc: 0.9445
Epoch 764/1000
24000/24000 - 1s - loss: 0.0041 - acc: 0.9992 - val_loss: 0.4697 - val_acc: 0.9442
Epoch 765/1000
24000/24000 - 1s - loss: 0.0026 - acc: 0.9996 - val_loss: 0.4699 - val_acc: 0.9441
Epoch 766/1000
24000/24000 - 1s - loss: 0.0031 - acc: 0.9996 - val_loss: 0.4669 - val_acc: 0.9451
Epoch 767/1000
24000/24000 - 1s - loss: 0.0056 - acc: 0.9991 - val_loss: 0.4640 - val_acc: 0.9450
Epoch 768/1000
24000

Epoch 842/1000
24000/24000 - 1s - loss: 0.0046 - acc: 0.9992 - val_loss: 0.4885 - val_acc: 0.9441
Epoch 843/1000
24000/24000 - 1s - loss: 0.0024 - acc: 0.9996 - val_loss: 0.6290 - val_acc: 0.9341
Epoch 844/1000
24000/24000 - 1s - loss: 0.0048 - acc: 0.9992 - val_loss: 0.4776 - val_acc: 0.9444
Epoch 845/1000
24000/24000 - 1s - loss: 0.0022 - acc: 0.9996 - val_loss: 0.4842 - val_acc: 0.9450
Epoch 846/1000
24000/24000 - 1s - loss: 0.0020 - acc: 0.9997 - val_loss: 0.4851 - val_acc: 0.9449
Epoch 847/1000
24000/24000 - 1s - loss: 0.0054 - acc: 0.9991 - val_loss: 0.4830 - val_acc: 0.9447
Epoch 848/1000
24000/24000 - 1s - loss: 0.0023 - acc: 0.9996 - val_loss: 0.4841 - val_acc: 0.9446
Epoch 849/1000
24000/24000 - 1s - loss: 0.0023 - acc: 0.9996 - val_loss: 0.4844 - val_acc: 0.9453
Epoch 850/1000
24000/24000 - 1s - loss: 0.0029 - acc: 0.9996 - val_loss: 0.4877 - val_acc: 0.9450
Epoch 851/1000
24000/24000 - 1s - loss: 0.0047 - acc: 0.9993 - val_loss: 0.4834 - val_acc: 0.9438
Epoch 852/1000
24000

Epoch 926/1000
24000/24000 - 1s - loss: 0.0018 - acc: 0.9997 - val_loss: 0.5044 - val_acc: 0.9442
Epoch 927/1000
24000/24000 - 1s - loss: 0.0028 - acc: 0.9995 - val_loss: 0.6318 - val_acc: 0.9357
Epoch 928/1000
24000/24000 - 1s - loss: 0.0068 - acc: 0.9989 - val_loss: 0.5077 - val_acc: 0.9439
Epoch 929/1000
24000/24000 - 1s - loss: 0.0034 - acc: 0.9994 - val_loss: 0.5039 - val_acc: 0.9445
Epoch 930/1000
24000/24000 - 1s - loss: 0.0027 - acc: 0.9996 - val_loss: 0.5036 - val_acc: 0.9442
Epoch 931/1000
24000/24000 - 1s - loss: 0.0024 - acc: 0.9996 - val_loss: 0.6588 - val_acc: 0.9357
Epoch 932/1000
24000/24000 - 1s - loss: 0.0055 - acc: 0.9991 - val_loss: 0.5028 - val_acc: 0.9446
Epoch 933/1000
24000/24000 - 1s - loss: 0.0019 - acc: 0.9997 - val_loss: 0.5183 - val_acc: 0.9443
Epoch 934/1000
24000/24000 - 1s - loss: 0.0055 - acc: 0.9989 - val_loss: 0.4954 - val_acc: 0.9439
Epoch 935/1000
24000/24000 - 1s - loss: 0.0021 - acc: 0.9997 - val_loss: 0.5050 - val_acc: 0.9449
Epoch 936/1000
24000